### df형태로 저장
- row: 인덱스
- column:
    1. 제품명
    2. 작성한 별점 _ 숫자만 따로 떼기
    3. 피부 타입 _ 태그 없으면 None으로 저장 (if문으로 거르기)
    4. 리뷰 _ &nbsp(\xa0) 문자는 띄어쓰기로 대체 (replace) <- 안해도 되네?
    
https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000161782&dispCatNo=100000100010008&trackingCd=Cat100000100010008_MID#

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup
import time

In [2]:
import pandas as pd
import numpy as np
import re
import requests

In [22]:
# 크롬 드라이버 설치 경로
chrome_driver = '/Users/ydj89/Downloads/chromedriver'

# 드라이버 객체 생성
driver = webdriver.Chrome(chrome_driver)

# df의 column
product_col = []
rating_col = []
skinType_col = []
review_col = []

# get함수에 이동하고자 할 사이트 전달 _ 제품 바뀔 때마다 업데이트 필요
url = 'https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000114129&dispCatNo=1000001000100090002&trackingCd=Cat1000001000100090002_Small&curation&egcode&rccode&egrankcode'
driver.get(url) 

# '리뷰'탭 클릭
review_button = driver.find_element_by_xpath('//*[@id="reviewInfo"]/a')
review_button.send_keys(Keys.ENTER)
    
# 리뷰개수 세기
stop = review_button.text
stop_review = int(re.sub(r'[^0-9]',"",stop))

# 리뷰 최대 1000개까지만 보여줌
if stop_review <= 1000:
    page_num = stop_review // 10
    if stop_review % 10 != 0:
        page_num += 1
        page_last = stop_review % 10
    else:
        page_last = 11
else:
    page_num = 100
    page_last = 11

end = 11

# (마지막 페이지 제외) 한 페이지당 리뷰 10개씩 보여줌
# 1~10페이지 까지는 태그 순서 1,2,3,...
# 이후부터는 [<<]버튼으로 인해 태그 순서 2,3,4... (뒤로 밀려남)
# 10페이지까지 긁었으면 [>>]버튼 클릭해서 다음 페이지로 넘어가야함
    
start = time.time()

# 수정사항: .click() 안먹는 경우도 있어서 .send_keys(Keys.ENTER)사용해서 클릭하기
for i in range(1, page_num+1):
    # 로딩될 때까지 대기
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#gdasList > li:nth-child(1) > div.review_cont > div.txt_inner')))
    # 페이지 소스 저장
    src = driver.page_source
    soup = BeautifulSoup(src)
        
    # 다음 페이지 선택(수정 후)
    if i >= 11:
        if i % 10 != 1:
            if i % 10 != 0:
                # 자꾸 elementinteractable 에러,,
                #page_button = driver.find_element_by_css_selector('#gdasContentsArea > div > div.pageing > a:nth-child({})'.format(i%10 + 1))
                page_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#gdasContentsArea > div > div.pageing > a:nth-child({})'.format(i%10 + 1))))
            else:
                #page_button = driver.find_element_by_css_selector('#gdasContentsArea > div > div.pageing > a:nth-child({})'.format(11))
                page_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#gdasContentsArea > div > div.pageing > a:nth-child({})'.format(11))))
        else:
            #page_button = driver.find_element_by_css_selector('#gdasContentsArea > div > div.pageing > a.next')
            page_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#gdasContentsArea > div > div.pageing > a.next')))
        page_button.send_keys(Keys.ENTER)
        
    elif i != 1:
        #page_button = driver.find_element_by_css_selector('#gdasContentsArea > div > div.pageing > a:nth-child({})'.format(i))
        page_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#gdasContentsArea > div > div.pageing > a:nth-child({})'.format(i))))
        page_button.send_keys(Keys.ENTER)        
    
    # 마지막 페이지는 리뷰가 10개까지 없을 수도
    if i == page_num:
        end = page_last
        
    for j in range(1, end):
        # 1. 제품명 (동일)
        product = soup.select_one('#Contents > div.prd_detail_box.renew > div.right_area > div > p.prd_name')
        product_col.append(product.get_text())

        # 2. 작성한 별점
        rating = soup.select_one('#gdasList > li:nth-child({}) > div.review_cont > div.score_area > span.review_point > span'.format(j))
            # 수정사항: 별점을 점수 '숫자'데이터만 저장하기
        rating_col.append(rating.get_text()[6])

        # 3. 피부 타입
        selector = '#gdasList > li:nth-child({}) > div.info > div > p.tag > em:nth-child(1) > span'.format(j)
        if not soup.select(selector):
            skinType_col.append('None')
        else:
            skinType = soup.select_one(selector)
            skinType_col.append(skinType.get_text())

        # 4. 리뷰
        review = soup.select('#gdasList > li:nth-child({}) > div.review_cont > div.txt_inner'.format(j))
        if not review:
            review_col.append('None')
        else:
            review_col.append(review[0].get_text())   

    # 디버깅용
    print(i, 'page')
    
finish = time.time()

# 브라우저 닫기
driver.close()

print(finish - start)

1 page
2 page
3 page
4 page
5 page
6 page
7 page
8 page
9 page
10 page
11 page
12 page
13 page
14 page
15 page
16 page
17 page
18 page
19 page
20 page
21 page
22 page
23 page
24 page
25 page
26 page
27 page
28 page
29 page
30 page
31 page
32 page
33 page
34 page
35 page
36 page
37 page
38 page
39 page
40 page
41 page
42 page
43 page
44 page
45 page
46 page
47 page
48 page
49 page
50 page
51 page
52 page


KeyboardInterrupt: 

In [4]:
print(len(review_col))
print(len(product_col))
print(len(rating_col))
print(len(skinType_col))

1000
1000
1000
1000


In [5]:
df = pd.DataFrame({
    'product_name' : product_col,
    'rating' : rating_col,
    'skin_type' : skinType_col,
    'review' : review_col
})

df

,product_name,rating,skin_type,review
0,[증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정),5,복합성,Dr.G 제품은 정말 사용해보시라고 추천드리고싶어요진짜 좋은 제품이거든요사춘기때 여...
1,[증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정),5,복합성,"👉🏻 간단정리! - good! : 자극없이 무난함, 봄여름가을에 적당한 수분감, 겨..."
2,[증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정),5,복합성,❤️나의 피부상태: 복합성+트러블 조금만 자극적이면 피부 뒤집어짐;💜장점: 여드름...
3,[증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정),5,지성,또 샀어요!!!! 진짜 닥터지 최고입니다.. 건성분들이 쓰시기에는 너무 빨리 날아가...
4,[증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정),5,복합성,#한줄요약가격: 화장품을 잘 안 쓰는 내 입장에서는 가성비 굿보습: 약간 건성~지성...
...,...,...,...,...
995,[증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정),5,건성,좋아용 계속 썼어용 아주 좋아용 좋아용 최고에용
996,[증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정),5,지성,수분감 촉촉하고좋아요 민감성인데 잘맞아요 만족합니다 ㅎㅎㅎㅎ
997,[증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정),4,각질,닥터지 블레미쉬 폼클렌징 사용했을때 트러블에 도움을 주길래 크림도 세일할때 사봤는데...
998,[증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정),5,복합성,매일 사용하는데 데일리로 너무 좋아요. 촉촉해요.


In [30]:
df.to_csv("test_review.csv")

## url 함수 따로, 긁어서 저장하는 함수 따로

In [5]:
def crawler():
    page = 1
    count=0
    global df_review
    global df1
    df_review = pd.DataFrame({
        'product_name' : ['제품이름'],
        'rating' : ['별점'],
        'skin_type' : ['피부타입'],
        'review' : ['리뷰']
    })
    while page < 23:
        print(page)

        url = 'https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=1000001000100090002&fltDispCatNo=&prdSort=01&pageIdx=' + str(page) + '&rowsPerPage=24&searchTypeSort=btn_thumb&plusButtonFlag=N&isLoginCnt=0&aShowCnt=0&bShowCnt=0&cShowCnt=0&trackingCd=Cat1000001000100090002_Small'
        headers = {"user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.109 Safari/537.36"}
                                 
        req = requests.get(url, headers=headers, verify=False)
        print(url)
        cont = req.content
        soup = BeautifulSoup(cont, 'html.parser')
 
    #제품 하나하나 하이퍼링크따서 selenium 함수에 넣기
        count = 0
        for urls in soup.select("div.prd_info > a"):
            try :
                if urls.attrs['href'].startswith("https://www.oliveyoung.co.kr/"):
                    print(urls.attrs['href'])
                    df1 = selenium(urls.attrs['href'])
                    df_review = pd.concat([df_review,df1])
                    
                    print("전체리뷰 수: ", len(df_review))
                    print(df_review)
                    
            except Exception as e:
                print("크롤러 에러 = ", end ="")
                print(e)
                continue
        page+=1

    return df_review

In [3]:
def selenium(url):
    # 드라이버 객체 생성
    driver = webdriver.Chrome(chrome_driver)
    
    # df의 column
    product_col = []
    rating_col = []
    skinType_col = []
    review_col = []

    # get함수에 이동하고자 할 사이트 전달
    driver.get(url) 

    # '리뷰'탭 클릭
    review_button = driver.find_element_by_xpath('//*[@id="reviewInfo"]/a')
    review_button.send_keys(Keys.ENTER)

    # 리뷰개수 세기
    stop = review_button.text
    stop_review = int(re.sub(r'[^0-9]',"",stop))

    # 리뷰 최대 1000개까지만 보여줌
    if stop_review <= 1000:
        page_num = stop_review // 10
        if stop_review % 10 != 0:
            page_num += 1
            page_last = stop_review % 10
        else:
            page_last = 11
    else:
        page_num = 100
        page_last = 11

    end = 11

    # (마지막 페이지 제외) 한 페이지당 리뷰 10개씩 보여줌
    # 1~10페이지 까지는 태그 순서 1,2,3,...
    # 이후부터는 [<<]버튼으로 인해 태그 순서 2,3,4... (뒤로 밀려남)
    # 10페이지까지 긁었으면 [>>]버튼 클릭해서 다음 페이지(_1페이지)로 넘어가야함
    

    # 수정사항: .click() 안먹는 경우도 있어서 .send_keys(Keys.ENTER)사용해서 클릭하기
    for i in range(1, page_num+1):
        
        # 로딩될 때까지 대기
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#gdasList > li:nth-child(1) > div.review_cont > div.txt_inner')))
        
        # 페이지 소스 저장
        src = driver.page_source
        soup = BeautifulSoup(src)

        # 다음 페이지 선택
        if i >= 11: # 11p~
            if i % 10 != 1:
                if i % 10 != 0: # _2p ~ _9p
                    # elementinteractable 에러,,
                    #page_button = driver.find_element_by_css_selector('#gdasContentsArea > div > div.pageing > a:nth-child({})'.format(i%10 + 1))
                    page_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#gdasContentsArea > div > div.pageing > a:nth-child({})'.format(i%10 + 1))))
                else: # 20p, 30p, ..., 100p
                    #page_button = driver.find_element_by_css_selector('#gdasContentsArea > div > div.pageing > a:nth-child({})'.format(11))
                    page_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#gdasContentsArea > div > div.pageing > a:nth-child({})'.format(11))))
            else: # 21p, 31p, ..., 91p
                #page_button = driver.find_element_by_css_selector('#gdasContentsArea > div > div.pageing > a.next')
                page_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#gdasContentsArea > div > div.pageing > a.next')))
            page_button.send_keys(Keys.ENTER)

        elif i != 1: # 2p ~ 9p
            #page_button = driver.find_element_by_css_selector('#gdasContentsArea > div > div.pageing > a:nth-child({})'.format(i))
            page_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#gdasContentsArea > div > div.pageing > a:nth-child({})'.format(i))))
            page_button.send_keys(Keys.ENTER)        

        # 마지막 페이지는 리뷰가 10개까지 없을 수도
        if i == page_num:
            end = page_last

        for j in range(1, end):
            # 1. 제품명 (동일)
            product = soup.select_one('#Contents > div.prd_detail_box.renew > div.right_area > div > p.prd_name')
            product_col.append(product.get_text())

            # 2. 작성한 별점
            rating = soup.select_one('#gdasList > li:nth-child({}) > div.review_cont > div.score_area > span.review_point > span'.format(j))
                # 수정사항: 별점을 점수 '숫자'데이터만 저장하기
            rating_col.append(rating.get_text()[6])

            # 3. 피부 타입
            selector = '#gdasList > li:nth-child({}) > div.info > div > p.tag > em:nth-child(1) > span'.format(j)
            if not soup.select(selector):
                skinType_col.append('None')
            else:
                skinType = soup.select_one(selector)
                skinType_col.append(skinType.get_text())

            # 4. 리뷰
            review = soup.select('#gdasList > li:nth-child({}) > div.review_cont > div.txt_inner'.format(j))
            if not review:
                review_col.append('None')
            else:
                review_col.append(review[0].get_text())   

        # 디버깅용
        print(i, 'page')

    df = pd.DataFrame({
      'product_name' : product_col,
      'rating' : rating_col,
      'skin_type' : skinType_col,
      'review' : review_col
    })

    # 브라우저 닫기
    driver.close()
    
    print('제품 별 긁어온 리뷰 페이지: ', page_num)
    
    return df

In [6]:
# 크롬 드라이버 설치 경로
chrome_driver = '/Users/ydj89/Downloads/chromedriver'

In [7]:
final_review = crawler()

1


/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.oliveyoung.co.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=1000001000100090002&fltDispCatNo=&prdSort=01&pageIdx=1&rowsPerPage=24&searchTypeSort=btn_thumb&plusButtonFlag=N&isLoginCnt=0&aShowCnt=0&bShowCnt=0&cShowCnt=0&trackingCd=Cat1000001000100090002_Small
https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000114129&dispCatNo=1000001000100090002&trackingCd=Cat1000001000100090002_Small&curation&egcode&rccode&egrankcode
1 page
2 page
3 page
4 page
5 page
6 page
7 page
8 page
9 page
10 page
11 page
12 page
13 page
14 page
15 page
16 page
17 page
18 page
19 page
20 page
21 page
22 page
23 page
24 page
25 page
26 page
27 page
28 page
29 page
30 page
31 page
32 page
33 page
34 page
35 page
36 page
37 page
38 page
39 page
40 page
41 page
42 page
43 page
44 page
45 page
46 page
47 page
48 page
49 page
50 page
51 page
52 page
53 page
54 page
55 page
56 page
57 page
58 page
59 page
60 page
61 page
62 page
63 page
64 page
65 page
66 page
67 page
68 page
69 page
7

89 page
90 page
91 page
92 page
93 page
94 page
95 page
96 page
97 page
98 page
99 page
100 page
제품 별 긁어온 리뷰 페이지:  100
전체리뷰 수:  3031
                                         product_name rating skin_type  \
0                                                제품이름     별점      피부타입   
0    [증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정)       5       복합성   
1    [증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정)       5       복합성   
2    [증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정)       5       복합성   
3    [증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정)       5        지성   
..                                                ...    ...       ...   
995  라로슈포제 시카플라스트 밤B5 100ml 기획(+시카 세럼 10ml,에빠끌라폼15ml)      5      None   
996  라로슈포제 시카플라스트 밤B5 100ml 기획(+시카 세럼 10ml,에빠끌라폼15ml)      3       복합성   
997  라로슈포제 시카플라스트 밤B5 100ml 기획(+시카 세럼 10ml,에빠끌라폼15ml)      5       복합성   
998  라로슈포제 시카플라스트 밤B5 100ml 기획(+시카 세럼 10ml,에빠끌라폼15ml)      5      None   
999  라로슈포제 시카플라스트 밤B5 100ml 기획(+시카 세럼 10ml,에빠끌라폼15ml)

1 page
2 page
3 page
4 page
5 page
6 page
7 page
8 page
9 page
10 page
11 page
12 page
13 page
14 page
15 page
16 page
17 page
18 page
19 page
20 page
21 page
22 page
23 page
24 page
25 page
26 page
27 page
28 page
29 page
제품 별 긁어온 리뷰 페이지:  29
전체리뷰 수:  5539
                                          product_name rating skin_type  \
0                                                 제품이름     별점      피부타입   
0     [증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정)       5       복합성   
1     [증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정)       5       복합성   
2     [증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정)       5       복합성   
3     [증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정)       5        지성   
..                                                 ...    ...       ...   
275  [특별기획]프리메라 알파인 베리 워터리 크림 50ml(+크림25ml+필링15ml+에...      4      None   
276  [특별기획]프리메라 알파인 베리 워터리 크림 50ml(+크림25ml+필링15ml+에...      5      None   
277  [특별기획]프리메라 알파인 베리 워터리 크림 50ml(+크림25ml+필링15ml+에...      5      

...

1 page
2 page
3 page
4 page
5 page
6 page
7 page
8 page
9 page
10 page
11 page
제품 별 긁어온 리뷰 페이지:  11
전체리뷰 수:  128026
                                         product_name rating skin_type  \
0                                                제품이름     별점      피부타입   
0    [증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정)       5       복합성   
1    [증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정)       5       복합성   
2    [증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정)       5       복합성   
3    [증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정)       5        지성   
..                                                ...    ...       ...   
101             케어존 더마 밀크 히알루로닉 크림 80ml (크림스킨31ml 증정)      5      None   
102             케어존 더마 밀크 히알루로닉 크림 80ml (크림스킨31ml 증정)      5      None   
103             케어존 더마 밀크 히알루로닉 크림 80ml (크림스킨31ml 증정)      5      None   
104             케어존 더마 밀크 히알루로닉 크림 80ml (크림스킨31ml 증정)      5      None   
105             케어존 더마 밀크 히알루로닉 크림 80ml (크림스킨31ml 증정)      5      None

60 page
61 page
62 page
63 page
64 page
65 page
제품 별 긁어온 리뷰 페이지:  65
전체리뷰 수:  129133
                                         product_name rating skin_type  \
0                                                제품이름     별점      피부타입   
0    [증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정)       5       복합성   
1    [증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정)       5       복합성   
2    [증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정)       5       복합성   
3    [증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정)       5        지성   
..                                                ...    ...       ...   
645                           브링그린 티트리 시카 수딩 크림 100ml      4      None   
646                           브링그린 티트리 시카 수딩 크림 100ml      4      None   
647                           브링그린 티트리 시카 수딩 크림 100ml      5       복합성   
648                           브링그린 티트리 시카 수딩 크림 100ml      5      None   
649                           브링그린 티트리 시카 수딩 크림 100ml      5      None   

                          

/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.oliveyoung.co.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=1000001000100090002&fltDispCatNo=&prdSort=01&pageIdx=22&rowsPerPage=24&searchTypeSort=btn_thumb&plusButtonFlag=N&isLoginCnt=0&aShowCnt=0&bShowCnt=0&cShowCnt=0&trackingCd=Cat1000001000100090002_Small
https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000144808&dispCatNo=1000001000100090002&trackingCd=Cat1000001000100090002_Small&curation&egcode&rccode&egrankcode
1 page
2 page
3 page
4 page
5 page
6 page
7 page
8 page
9 page
10 page
제품 별 긁어온 리뷰 페이지:  10
전체리뷰 수:  129346
                                        product_name rating skin_type  \
0                                               제품이름     별점      피부타입   
0   [증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정)       5       복합성   
1   [증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정)       5       복합성   
2   [증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정)       5       복합성   
3   [증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정)       5        지성 

1 page
2 page
3 page
4 page
5 page
6 page
7 page
8 page
9 page
10 page
11 page
12 page
13 page
14 page
15 page
16 page
17 page
18 page
19 page
20 page
21 page
22 page
23 page
24 page
25 page
26 page
27 page
28 page
29 page
30 page
31 page
32 page
33 page
34 page
35 page
제품 별 긁어온 리뷰 페이지:  35
전체리뷰 수:  130119
                                         product_name rating skin_type  \
0                                                제품이름     별점      피부타입   
0    [증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정)       5       복합성   
1    [증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정)       5       복합성   
2    [증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정)       5       복합성   
3    [증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정)       5        지성   
..                                                ...    ...       ...   
337                               닥터디퍼런트 비타리프트 에이 20g      4      None   
338                               닥터디퍼런트 비타리프트 에이 20g      5      None   
339                       

In [8]:
len(final_review)

130456

In [10]:
final_review.to_csv("reviews.csv", encoding='utf-8')

In [21]:
final_review = pd.read_csv("reviews.csv")

In [22]:
final_review.head()

,Unnamed: 0,product_name,rating,skin_type,review
0,0,제품이름,별점,피부타입,리뷰
1,0,[증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정),5,복합성,Dr.G 제품은 정말 사용해보시라고 추천드리고싶어요진짜 좋은 제품이거든요사춘기때 여...
2,1,[증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정),5,복합성,"👉🏻 간단정리! - good! : 자극없이 무난함, 봄여름가을에 적당한 수분감, 겨..."
3,2,[증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정),5,복합성,❤️나의 피부상태: 복합성+트러블 조금만 자극적이면 피부 뒤집어짐;💜장점: 여드름...
4,3,[증량]닥터지 레드 블레미쉬 클리어 수딩크림 기획 (본품70ml+30ml 추가 증정),5,지성,또 샀어요!!!! 진짜 닥터지 최고입니다.. 건성분들이 쓰시기에는 너무 빨리 날아가...


In [23]:
final_review.tail()

,Unnamed: 0,product_name,rating,skin_type,review
130451,82,밀크터치 서양송악 진정크림+송악점보패드(60매),5,None,최애 입니다이제 이것만 쓸거예요또사러 가야지 눈누난나 신난다
130452,83,밀크터치 서양송악 진정크림+송악점보패드(60매),5,None,자극이 없고 일단은 너무 촉촉해서 좋아요!이것만 쓰게 될것같아ㅛ
130453,84,밀크터치 서양송악 진정크림+송악점보패드(60매),5,None,배송도 빠르고 세일할 때 사야할거 같아서 샀어요송악크림 건성한테 정말 좋을거같아요
130454,85,밀크터치 서양송악 진정크림+송악점보패드(60매),5,None,가벼운제형이고 끈적거리지 않아서 좋았어요. 그리고 진정에도 좋습니다!
130455,86,밀크터치 서양송악 진정크림+송악점보패드(60매),5,None,처음에 써보는 제품인데 의외로 괜찮아요. 보습과 진정효과가 좋습니다
